# Beispiel-Implementierung: Lokale Suchmaschine

## Ziel der Beispiel-Implementierung
Im Folgenden wird eine Anwendung der zuvor theoretisch diskutierten Inhalte vorgestellt. Dabei soll eine lokale Suchmaschine entwickelt werden, welche in der Lage ist, pdf-Dateien auf einem lokalen Computer-System zu parsen, in einen invertierten Index aufzunehmen sowie Suchanfragen eines Benutzers sinnvoll zu beantworten. <br>
Zur Relevanz-Bestimmung der Dokumente wird das TF-IDF-Maß, welches bereits vorgestellt wurde, genutzt. Um den Index zu speichern, wird die von Python mitgelieferte Datenstruktur "dictionary", welche im Grunde eine Hashmap ist, genutzt.
Weiter werden einige Bibliotheken eingesetzt, welche einige Vorarbeit leisten und damit den Code der Beispiel-Implementierung auf das Wesentliche beschränken. So soll die grundlegende Arbeitsweise eines Information Retrieval-Systems dargelegt werden.

## Genutzte Bibliotheken
Bevor mit der eigentlichen Implementierung der lokalen Suchmaschine begonnen werden kann, müssen einige Bibliotheken eingebunden werden. Darunter fallen Apache Tika, das Math-Modul von Python, os (um auf die Directories zugreifen zu können), python-magic, regular expressions (re) (und noch weitere, bei Bedarf einfügen!). <br>

### Tika
Tika liefert eine Parser, mit dessen Hilfe der Text aus - unter anderem - pdf-Dateien extrahiert werden kann.
Mit dem Aufruf _parser.from_\__file(file)_ kann eine pdf-Datei in reinen Text umgewandelt werden. Die Funktion liefert ein Dictionary zurück, welches einen Key _content_ besitzt, über den auf den Inhalt der pdf-Datei zugegriffen werden kann.

### python-magic
Mittels python-magic ist es möglich, unabhängig von der Dateiendung, den Typ einer Datei zu ermitteln. Dies hat den Vorteil, dass die Suchmaschine sowohl unter Windows, als auch unter Unix-Systemen, alle pdf-Dateien finden kann, da unter Unix die Dateiendung keine garantierten Rückschlüsse auf den Typ der Datei zulässt.


In [100]:
from tika import parser
import magic
import math
import os
import string
import re
from nltk.tokenize import RegexpTokenizer

## Der Index
Nachdem die benötigten Bibliotheken bekannt sind, kann der Index implementiert werden. Bevor dieser jedoch aufgebaut werden kann, sind einige Vorarbeiten nötig, die durch die vorgestellten Bibliotheken gestützt werden.
Der Index wird im Folgenden als Klasse implementiert. Diese beinhaltet die folgenden Methoden, die in den folgenden Abschnitten genauer diskutiert werden:
- buildIndex()
- retrieve()
- calcTFIDF()

Weiter werden die folgenden Member-Variablen benötigt:
- hashmap
- fileCount


In [104]:
class Index:
    hashmap = {} #dictionary
    fileCount = 0 #integer, Gesamtzahl aller gefunden Dateien

## buildIndex
Die Methode _buildIndex_ baut - wie der Name bereits vermuten lässt - den Index auf. Dabei dient ein Dictionary als Basis-Datenstruktur. Zudem führt diese Methode die Verarbeitung der pdf-Dateien mittels Apache tika und python-magic durch.

Der erste Schritt stellt das Iterieren über alle Directories dar. Gestartet wird bei Linux-Systemen im Root-Directory, unter Windows-Systemen muss über jede Partition iteriert werden.

In [105]:
def buildIndex(self):
    # alle Start-Verzeichnisse holen
    #start = self.__getStartDirectories()
    start = ["C:/Users/jonas/Uni/Praxisbericht/Praxisbericht 5. Semester/Latex"]
    # Magic-Instanz erstellen, um Datei-Typ bestimmen zu können
    mime = magic.Magic(mime=True)
    
    for s in start:
        for subdir, dir, files in os.walk(s):
            for f in files:
                if mime.from_file(s+"/"+f) == "application/pdf":
                    # in Text umwawndeln und tokenization durchführen
                    fileData = parser.from_file(s+"/"+f)
                    rawText = fileData['content']
                    
                    self.__preprocessText(rawText)
                    
    return

Index.buildIndex = buildIndex

### Hilfsmethoden
In diesem Abschnitt werden die genutzten Hilfsmethoden kurz vorgestellt. Diese werden jedoch nicht in der Tiefe behandelt, wie die drei Haupt-Methoden behandelt werden. 

#### __getStartDirectories
Die Methode _\_\_getSartDirectories_ liefert eine Liste zurück, welche abhängig vom Betriebssystem, auf dem die Suchmaschine läuft, die Start-Verzeichnisse zurückgibt, in denen nach pdf-Dateien gesucht werden soll. Falls das zugrunde liegende Betriebssystem ein Linux-basiertes System ist, wird die Liste __["/"]__ zurückgegeben, falls ein Windows-System zugrundeliegt, wird die Liste aller Partitionen zurückgegeben.

In [106]:
def __getStartDirectories(self):
    start = []
    
    if platform.system() == "Linux":
        start.append("/")
    elif platform.system() == "Windows":
        start = ['%s:' % d for d in string.ascii_uppercase if os.path.exists('%s:' % d)]
    else:
        raise EnvironmentError
        
    return start

Index.__getStartDirectories = __getStartDirectories

#### __addToIndex

Diese Methode bekommt als Argument einen String, welcher den von tika extrahierten Text enthält. Dieser String wird mithilfe der in Python enthaltenen Methode _split_ aufgeteilt und in einer Liste zusammengefasst. Weiter werden Satzzeichen wie Punkte, Kommata, etc. aus dem String bzw. der Liste herausgefiltert.

In [107]:
def __addToIndex(self, text):
    # string zu Liste machen, dabei Punkt, Komma etc. rauswerfen
    forbidden = ".,?!'\";:-"
    textList = [word for word in text.split() if word not in forbidden]
    print(textList[:100])
    # Post-processing der Wort-Liste
    
    
Index.__addToIndex = __addToIndex

In [160]:
def __preprocessText(self, txt):
    # lower all:
    txt = txt.lower()
    txt = re.sub(r'\d+', '', txt)
    intermediate = txt.split('\n')
    toTokenize = ""
    for line in intermediate:
        if line != '' and line[-1] == "-":
            i = intermediate.index(line)
            nextWords = intermediate[i+1].split()
            newLine = line + nextWords[0]
            newLine = newLine[1:]
            intermediate[i+1] = "".join(newLine)
    tokenizer = RegexpTokenizer(r'\w+')
    result = tokenizer.tokenize(toTokenize)
    print(result[:100])
    return result
    
Index.__preprocessText = __preprocessText

In [161]:
ind = Index()
ind.buildIndex()

IndexError: list index out of range